<a href="https://colab.research.google.com/github/Aniket21628/AI_Support/blob/main/Pokemon_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install -q langchain langchain-nvidia-ai-endpoints

In [3]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color = "#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [14]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from google.colab import userdata

api_key = userdata.get('NVIDIA_API_KEY')

In [5]:
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface = "State: "):
  def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
  return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

In [ ]:
pip install langchain-community wikipedia

In [ ]:
from langchain_community.document_loaders import WikipediaLoader
from datetime import datetime, timedelta
import os
import json

docs = WikipediaLoader(query="Pokemon and Everything about Pokemon", load_max_docs=1000).load()


In [11]:
def create_chunks_with_headers(doc, doc_index):
    chunk_size = 800
    chunk_overlap = 100
    chunks = []
    start = 0
    doc_content = doc.page_content
    doc_length = len(doc.page_content)

    while start < doc_length:
        end = min(start + chunk_size, doc_length)
        chunk = doc_content[start:end]

        if start != 0:
            chunk = doc_content[max(start - chunk_overlap, 0):end]

        chunk_json = {
            "meta_data": {
                "title": doc.metadata["title"],
                "summary": doc.metadata['summary'],
                "source_url": doc.metadata['source'],
            },
            "chunk_index": len(chunks) + 1,
            "content": chunk
        }
        chunks.append(chunk_json)

        start += chunk_size

    return chunks

In [ ]:
all_chunks = []

for i,doc in enumerate(docs):
  chunks = create_chunks_with_headers(doc, i+1)
  all_chunks.extend(chunks)
  print(f"Created {len(chunks)} chunks for document {i+1}")

print("All data has been processed")

In [ ]:
%pip install --upgrade --quiet lark langchain-chroma
%pip install --upgrade --quiet langchain-cohere

In [18]:
from langchain_core.documents import Document
from langchain_chroma import Chroma
from langchain_cohere import CohereEmbeddings
from google.colab import userdata
import time

# Get API key
cohere_api_key = userdata.get('COHERE_API_KEY')

# Initialize embeddings with model
embeddings_model = CohereEmbeddings(
    cohere_api_key=cohere_api_key,
    model="embed-english-v3.0"
)

# Convert chunks to Document objects
def convert_chunks_to_documents(chunks):
    return [
        Document(page_content=chunk["content"], metadata=chunk["meta_data"])
        for chunk in chunks
    ]

# Batching and backoff wrapper
def embed_documents_in_batches(documents, batch_size=25, delay=65):
    vectorstores = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        print(f"Embedding batch {i // batch_size + 1}/{(len(documents) + batch_size - 1) // batch_size}")
        try:
            vs = Chroma.from_documents(batch, embeddings_model)
            vectorstores.append(vs)
        except Exception as e:
            print(f"Failed batch {i // batch_size + 1}: {e}")
            print("Waiting before retry...")
            time.sleep(delay)
    return vectorstores

# === Main processing ===
all_chunks = []

# Example: Suppose `docs` is your list of Wikipedia documents
for i, doc in enumerate(docs):
    chunks = create_chunks_with_headers(doc, i)
    all_chunks.extend(chunks)

documents = convert_chunks_to_documents(all_chunks)

# Now embed in safe batches
vectorstores = embed_documents_in_batches(documents)


Embedding batch 1/90
Embedding batch 2/90
Embedding batch 3/90
Embedding batch 4/90
Embedding batch 5/90
Embedding batch 6/90
Embedding batch 7/90
Embedding batch 8/90
Embedding batch 9/90
Embedding batch 10/90
Embedding batch 11/90
Embedding batch 12/90
Embedding batch 13/90
Embedding batch 14/90
Embedding batch 15/90
Embedding batch 16/90
Embedding batch 17/90
Embedding batch 18/90
Embedding batch 19/90
Embedding batch 20/90
Embedding batch 21/90


Embedding batch 22/90


Failed batch 22: status_code: 429, body: {'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Waiting before retry...
Embedding batch 23/90
Embedding batch 24/90
Embedding batch 25/90
Embedding batch 26/90
Embedding batch 27/90
Embedding batch 28/90
Embedding batch 29/90
Embedding batch 30/90
Embedding batch 31/90
Embedding batch 32/90
Embedding batch 33/90
Embedding batch 34/90
Embedding batch 35/90
Embedding batch 36/90
Embedding batch 37/90
Embedding batch 38/90
Embedding batch 39/90
Embedding batch 40/90
Embedding batch 41/90
Embedding batch 42/90


Embedding batch 43/90


Failed batch 43: status_code: 429, body: {'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Waiting before retry...
Embedding batch 44/90
Embedding batch 45/90
Embedding batch 46/90
Embedding batch 47/90
Embedding batch 48/90
Embedding batch 49/90
Embedding batch 50/90
Embedding batch 51/90
Embedding batch 52/90
Embedding batch 53/90
Embedding batch 54/90
Embedding batch 55/90
Embedding batch 56/90
Embedding batch 57/90
Embedding batch 58/90
Embedding batch 59/90
Embedding batch 60/90
Embedding batch 61/90
Embedding batch 62/90
Embedding batch 63/90


Embedding batch 64/90


Failed batch 64: status_code: 429, body: {'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Waiting before retry...
Embedding batch 65/90
Embedding batch 66/90
Embedding batch 67/90
Embedding batch 68/90
Embedding batch 69/90
Embedding batch 70/90
Embedding batch 71/90
Embedding batch 72/90
Embedding batch 73/90
Embedding batch 74/90
Embedding batch 75/90
Embedding batch 76/90
Embedding batch 77/90
Embedding batch 78/90
Embedding batch 79/90
Embedding batch 80/90
Embedding batch 81/90
Embedding batch 82/90
Embedding batch 83/90
Embedding batch 84/90


Embedding batch 85/90


Failed batch 85: status_code: 429, body: {'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}
Waiting before retry...
Embedding batch 86/90
Embedding batch 87/90
Embedding batch 88/90
Embedding batch 89/90
Embedding batch 90/90
